In [1]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne
import json
from PIL import Image
from random import shuffle

In [5]:
def written():
    #get reference answer
    fin = open('result','r')
    dictionary = fin.readlines()
    fin.close()
    os.chdir('learning_set/')
    dictionary = json.loads(dictionary[0])
    
    #create X_train, y_train
    #get data from one image
    def dataImage(file,y,dictionary):
        image = Image.open(file)
        y.append(dictionary[file])
        data = np.asarray(image)
        #image black-white, so tuple (a,a,a), and we would take one, min is first find varient
        data = data.min(axis=2)
        image.close()
        return np.array([data/np.float32(256)])
    
    yTrain = []
    l = os.listdir()
    shuffle(l)
    files = [file for file in l if file != 'result']
    xTrain = np.array([dataImage(file,yTrain,dictionary) for file in files])
    
    
    os.chdir('..')
    fin = open('test_result','r')
    dictionary = fin.readlines()
    fin.close()
    dictionary = json.loads(dictionary[0])
    yTest = []
    os.chdir('test_set')

    files = [file for file in os.listdir() if file != 'result']
    xTest = np.array([dataImage(file,yTest,dictionary) for file in files])
    
    
    xTrain, xVal = xTrain[:-1500], xTrain[-1500:]
    yTrain, yVal = yTrain[:-1500], yTrain[-1500:]
    
    return xTrain, np.asarray(yTrain), xVal, np.asarray(yVal) , xTest, np.asarray(yTest)

In [6]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(None, 1, 38, 38),
                                        input_var=input_var)
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    #nonlinearity indicate function activitaion
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.3),
            num_units = 256,
            nonlinearity=lasagne.nonlinearities.rectify)
    
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.3),
            num_units=3,
            nonlinearity=lasagne.nonlinearities.softmax)
    
    return network

In [7]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [8]:
xTrain, yTrain, xVal, yVal, xTest, yTest = written()
#yTrain = np.array([np.fromstring(x, dtype=np.uint8, sep=' ')[0] for x in yTrain])
#yVal = np.array([np.fromstring(x, dtype=np.uint8, sep=' ')[0] for x in yVal])
#yTest = np.array([np.fromstring(x, dtype=np.uint8, sep=' ')[0] for x in yTest])

In [9]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
network = build_cnn(input_var)
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,target_var)
test_loss = test_loss.mean()

test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),dtype=theano.config.floatX)

train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
test_fn = theano.function([input_var,target_var], [test_loss, test_acc, prediction])

In [10]:
def runO(num_epochs):
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(xTrain, yTrain, 500, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(xVal, yVal, 500, shuffle=False):
            print(1)
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

    test_err = 0
    test_acc = 0
    test_batches = 0
    res = []
    for batch in iterate_minibatches(xTest, yTest, 500, shuffle=False):
        inputs, targets = batch
        err, acc, result = test_fn(inputs, targets)
        res.append(result)
        test_err += err
        test_acc += acc
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
    print("  test accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))
    return res

In [12]:
res = runO(100)

1
1
1
Epoch 1 of 100 took 67.556s
  training loss:		0.981518
  validation loss:		0.861606
  validation accuracy:		51.87 %
1
1
1
Epoch 2 of 100 took 80.455s
  training loss:		0.874332
  validation loss:		0.733581
  validation accuracy:		58.87 %
1
1
1
Epoch 3 of 100 took 80.358s
  training loss:		0.841480
  validation loss:		2.102430
  validation accuracy:		40.60 %
1
1
1
Epoch 4 of 100 took 81.210s
  training loss:		0.892183
  validation loss:		0.781412
  validation accuracy:		60.20 %
1
1
1
Epoch 5 of 100 took 81.376s
  training loss:		0.739979
  validation loss:		0.507732
  validation accuracy:		83.20 %
1
1
1
Epoch 6 of 100 took 80.584s
  training loss:		0.713027
  validation loss:		0.537877
  validation accuracy:		74.60 %
1
1
1
Epoch 7 of 100 took 80.290s
  training loss:		0.833793
  validation loss:		0.631700
  validation accuracy:		72.20 %
1
1
1
Epoch 8 of 100 took 80.521s
  training loss:		1.044139
  validation loss:		0.780635
  validation accuracy:		67.73 %
1
1
1
Epoch 9 of 100 too

In [19]:
np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
#And load them again later on like this:
with np.load('model.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(network, param_values)